In [1]:
import os 
os.chdir('../../')
print("Current working directory is now: ", os.getcwd())

import pandas as pd 
import numpy as np
import csv
from sklearn.linear_model import LogisticRegression

import FL_utils.FL_interpretable_functions as interpret
import FL_utils.FL_RiskSLIM as slim
import FL_utils.FL_stumps as stumps

Current working directory is now:  C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\kentucky\ky_fl_combined_test\FL Model


C:\Users\binha\Anaconda3\lib\site-packages\sklearn\externals\six.py:31: DeprecationWarning:

The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).



### EBM & CART

In [2]:
KY_data = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined_test/data preparation/KY_cleaned").sort_values('person_id')
FL_data = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined_test/data preparation/FL_cleaned").sort_values('person_id')

## split x and y 
KY_X = KY_data.loc[:,:'current_violence20']
KY_Y = KY_data['felony_two_year'].values
FL_X = FL_data.loc[:,:'current_violence20']
FL_Y = FL_data['felony_two_year'].values

In [3]:
#### CART
depth = [1,2,3,4,5]
impurity = [0.001, 0.002, 0.003, 0.004, 0.005]
cart_summary = interpret.CART(KY_X, KY_Y, FL_X, FL_Y, depth, impurity, seed=816)

### EBM
estimators = [40,60,80,100]
depth = [1,2,3]
learning_rate = [0.01]
holdout_split = [0.7, 0.9]
ebm_summary = interpret.EBM(KY_X, KY_Y, FL_X, FL_Y, learning_rate, depth, estimators, holdout_split, seed=816)

In [5]:
np.mean(cart_summary['KY_score']), np.mean(ebm_summary['KY_score'])

(0.5807815743244424, 0.7053571456696053)

### Lasso Stumps

In [4]:
KY_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined_test/data preparation/KY_stumps").sort_values('person_id')
FL_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined_test/data preparation/FL_stumps").sort_values('person_id')

## split x and y 
KY_X = KY_stumps.loc[:,:'current_violence201']
KY_Y = KY_stumps['felony_two_year'].values
FL_X = FL_stumps.loc[:,:'current_violence201']
FL_Y = FL_stumps['felony_two_year'].values

## columns 
cols = KY_X.columns[1:]

In [5]:
single_stump_model = stumps.stump_model(KY_X, 
                                        KY_Y, 
                                        FL_X, 
                                        FL_Y, 
                                        0.1, 
                                        cols, 
                                        816)
single_stump_model['features']

['age_at_current_charge21',
 'age_at_current_charge30',
 'p_arrest1',
 'p_arrest10',
 'p_charges1',
 'p_charges2',
 'p_charges5',
 'p_violence2',
 'p_felony2',
 'p_misdemeanor3',
 'p_property3',
 'p_felprop_viol1',
 'p_drug3',
 'p_dui1',
 'p_pending_charge5',
 'p_probation3',
 'p_incarceration1',
 'three_year1']

In [6]:
c_grid={'C': [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1]}
stumps_summary = stumps.stump_cv(KY_X, KY_Y, FL_X, FL_Y, cols, c_grid, seed=816)

In [7]:
stumps_summary['best_params'], np.mean(stumps_summary['KY_score']), np.mean(stumps_summary['auc_diff'])

([{'C': 0.08}, {'C': 0.07}, {'C': 0.1}, {'C': 0.09}, {'C': 0.1}],
 0.6702099125866037,
 0.046871895892648376)

### RiskSLIM

In [37]:
single_stump_model = stumps.stump_model(KY_X, 
                                        KY_Y, 
                                        FL_X, 
                                        FL_Y, 
                                        0.12,  
                                        cols, 816)
selected_features = single_stump_model['features']
len(selected_features)

21

In [38]:
### Subset features
sub_FL_X = FL_stumps.loc[:, selected_features]
sub_KY_X = KY_stumps.loc[:, selected_features]
sub_FL_X.insert(0, '(Intercept)', 1)

In [39]:
riskslim_summary = slim.risk_cv(sub_KY_X, 
                                KY_Y, 
                                sub_FL_X, 
                                FL_Y, 
                                y_label = 'felony_two_year', 
                                max_coef = 5, 
                                max_coef_number = 5, 
                                max_runtime=1000,
                                max_offset=100,
                                c=1e-4,  
                                seed=816)

setting c0 = 0.0 to ensure that intercept is not penalized
11/28/19 @ 01:46 PM | 263 rows in lookup table
11/28/19 @ 01:46 PM | ------------------------------------------------------------
11/28/19 @ 01:46 PM | runnning initialization procedure
11/28/19 @ 01:46 PM | ------------------------------------------------------------
11/28/19 @ 01:46 PM | CPA produced 2 cuts
11/28/19 @ 01:46 PM | running naive rounding on 22 solutions
11/28/19 @ 01:46 PM | best objective value: 0.4524
11/28/19 @ 01:46 PM | rounding produced 5 integer solutions
11/28/19 @ 01:46 PM | best objective value is 0.4517
11/28/19 @ 01:46 PM | running sequential rounding on 22 solutions
11/28/19 @ 01:46 PM | best objective value: 0.4524
11/28/19 @ 01:46 PM | sequential rounding produced 6 integer solutions
11/28/19 @ 01:46 PM | best objective value: 0.4478
11/28/19 @ 01:46 PM | polishing 11 solutions
11/28/19 @ 01:46 PM | best objective value: 0.4478
11/28/19 @ 01:46 PM | polishing produced 5 integer solutions
11/28/19 

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/28/19 @ 01:46 PM | adding 210 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.4478.
Tried aggregator 1 time.
Reduced MIP has 23 rows, 46 columns, and 87 nonzeros.
Reduced MIP has 21 binaries, 23 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.05 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.03 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/28/19 @ 01:46 PM | adding 251 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.4548.
Tried aggregator 1 time.
Reduced MIP has 23 rows, 46 columns, and 87 nonzeros.
Reduced MIP has 21 binaries, 23 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.05 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.03 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

 128050 42064        0.4469     9        0.4510        0.4348   706850    3.60%           rho_5 D 128050 128049     30
 130304 43021        0.4351    23        0.4510        0.4348   722606    3.60%           rho_7 D 130304 130303     20
 132891 43854        0.4350    13        0.4510        0.4350   735443    3.56%           rho_6 U 132891 132839     16
 135694 44774        cutoff              0.4510        0.4351   746905    3.53%           rho_7 N 135694 135693     20
 138532 45609        0.4353    17        0.4510        0.4353   757528    3.50%           rho_4 D 138532 138531     31
 141215 46323        0.4423    11        0.4510        0.4354   767885    3.48%           rho_5 D 141215 141214     29
 143992 47070        0.4377    16        0.4510        0.4355   779154    3.45%           rho_3 D 143992 143991     20
Elapsed time = 23.95 sec. (49245.29 ticks, tree = 14.28 MB, solutions = 6)
 146444 47689        0.4355    18        0.4510        0.4355   790428    3.44%           rh

 419797 42729        cutoff              0.4508        0.4447  1905014    1.35%          rho_19 U 419797 419795     24
 429144 38660        cutoff              0.4508        0.4452  1939825    1.25%          rho_12 U 429144 344088     32
Elapsed time = 71.75 sec. (163915.24 ticks, tree = 15.40 MB, solutions = 8)
 438293 33961        cutoff              0.4508        0.4458  1972511    1.13%           rho_2 U 438293  25533     28
 447221 29029        cutoff              0.4508        0.4463  2004406    1.01%          rho_17 U 447221 159928     24
 456407 24588        0.4489     6        0.4508        0.4469  2034194    0.88%           rho_2 D 456407 456405     28
 465560 19394        cutoff              0.4508        0.4476  2062595    0.73%          rho_17 U 465560 126970     39
 474401 12797        cutoff              0.4508        0.4484  2089633    0.54%           rho_4 U 474401 452394     15
 483731  6299        0.4495    13        0.4508        0.4494  2114832    0.32%           r

...

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/28/19 @ 01:54 PM | adding 248 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.4403.
Tried aggregator 1 time.
Reduced MIP has 23 rows, 46 columns, and 87 nonzeros.
Reduced MIP has 21 binaries, 23 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.05 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.03 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

Elapsed time = 19.00 sec. (37662.65 ticks, tree = 11.54 MB, solutions = 10)
 115832 37099        0.4255    13        0.4377        0.4221   663603    3.55%          rho_14 D 115832 115831     22
 118325 37889        0.4273     9        0.4377        0.4221   679031    3.55%           rho_2 D 118325 118324     31
 120830 38626        0.4305    11        0.4377        0.4221   693436    3.55%          rho_13 D 120830 120829     27
 123341 39439        cutoff              0.4377        0.4221   708633    3.55%           rho_7 U 123341 114917     27
 125615 40146        0.4315     7        0.4377        0.4221   724159    3.55%          rho_13 D 125615 125613     26
 127950 40823        0.4278     8        0.4377        0.4221   739361    3.55%          rho_14 D 127950 127949     27
 130460 41559        0.4230    13        0.4377        0.4221   754240    3.55%          rho_14 D 130460 130459     24
 132950 42279        0.4332     7        0.4377        0.4221   768364    3.55%          rh

 383168 62337        cutoff              0.4377        0.4291  1858658    1.95%           rho_5 U 383168 230336     22
 392638 59986        cutoff              0.4377        0.4294  1898685    1.88%           rho_7 U 392638 163740     17
 402027 57242        cutoff              0.4377        0.4298  1938510    1.79%          rho_10 U 402027 358251     27
 411309 54757        0.4305     9        0.4377        0.4302  1977847    1.71%           rho_7 U 411309 410547     21
 420554 51828        cutoff              0.4377        0.4305  2018069    1.63%          rho_14 U 420554 332104     22
 429783 48845        cutoff              0.4377        0.4309  2056181    1.54%           rho_2 U 429783 242191     26
Elapsed time = 71.49 sec. (164522.33 ticks, tree = 17.96 MB, solutions = 10)
 438580 44945        cutoff              0.4377        0.4313  2094276    1.44%           rho_2 U 438580 318732     27
 447688 41201        cutoff              0.4377        0.4318  2130952    1.34%           

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/28/19 @ 01:56 PM | adding 241 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.4458.
Tried aggregator 1 time.
Reduced MIP has 23 rows, 46 columns, and 87 nonzeros.
Reduced MIP has 21 binaries, 23 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.05 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.03 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

ValueError: max() arg is an empty sequence

In [40]:
np.mean(riskslim_summary['FL_validation']), np.mean(riskslim_summary['KY_score'])

(0.5679234841807851, 0.6184414050632507)

In [41]:
#### save results
summary_felony2_fl_inter_model = {"CART": cart_summary,
                                  "EBM": ebm_summary,
                                  "Stumps": stumps_summary,
                                  "RiskSLIM": riskslim_summary}

In [42]:
path = "C:\\Users\\binha\\Documents\\Duke\\Cynthia Research\\KY-analysis-mytrials\\KY Recidivism\\KY Results\\KY_FL\\FL Model\\interpretable\\Two Year\\"
results = [["Felony",             
            np.str(round(np.mean(cart_summary['KY_score']),3)) + " (" + np.str(round(np.std(cart_summary['KY_score']),3)) + ")", 
            np.str(round(np.mean(ebm_summary['KY_score']), 3)) + " (" + np.str(round(np.std(ebm_summary['KY_score']),3)) + ")", 
            np.str(round(np.mean(stumps_summary['KY_score']),3)) + " (" + np.str(round(np.std(stumps_summary['KY_score']),3)) + ")", 
            np.str(round(np.mean(riskslim_summary['KY_score']), 3)) + " (" + np.str(round(np.std(riskslim_summary['KY_score']),3)) + ")"]]
with open(path + 'Two Year FL_inter_model.csv', 'a') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerows(results)